In [2]:
import os
from configparser import ConfigParser

config = ConfigParser()
config.read('../conf.ini')
openai_token=config['secret']['openai_token']
os.environ['OPENAI_API_KEY'] = openai_token

# Chains
- 좀 더 복잡한 Application을 구동하기 위해서 사용
    - 구성 요소를 체인으로 함께 구성
    - Application을 디버스, 유지 관리가 훨씬 편해짐

In [4]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["company", "product"],
    template="Can you recommend 5 good names for {company} that makes {product}?",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run({
    'company': "ABC Startup",
    'product': "colorful socks"
    }))



1. Joyful Socks
2. Colorful Crew
3. Rainbow Hues
4. Bright Feet
5. Happy Toes


## How to

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(temperature=0)
prompt_template = "Tell me a {adjective} joke"
llm_chain = LLMChain(
    llm=chat, 
    prompt=PromptTemplate.from_template(prompt_template), 
    verbose=True # For Debugging
    )

llm_chain(inputs={"adjective": "corny"})
llm_chain.output_keys



> Entering new LLMChain chain...
Prompt after formatting:
Tell me a corny joke

> Finished chain.


['text']

In [7]:
# 모두 같은 명령어
llm_chain.run({"adjective": "corny"})
llm_chain.run("corny")

llm_chain("corny")
llm_chain({"adjective": "corny"})



> Entering new LLMChain chain...
Prompt after formatting:
Tell me a corny joke

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Tell me a corny joke

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Tell me a corny joke

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Tell me a corny joke

> Finished chain.


{'adjective': 'corny',
 'text': "Sure, here's a corny joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"}

### OpenAI Function

#### Structued Output

In [10]:
from pydantic import BaseModel, Field
from typing import Optional, Sequence

class Person(BaseModel):
    """Identifying information about a person."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")
    
class People(BaseModel):
    """Identifying information about all people in a text."""

    people: Sequence[Person] = Field(..., description="The people in the text")

In [13]:
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage

# If we pass in a model explicitly, we need to make sure it supports the OpenAI function-calling API.
llm = ChatOpenAI(model="gpt-4", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world class algorithm for extracting information in structured formats."),
        ("human", "Use the given format to extract information from the following input: {input}"),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

chain = create_structured_output_chain(Person, llm, prompt, verbose=True)
chain.run("Sally is 13 and favorite food is Korean food")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for extracting information in structured formats.
Human: Use the given format to extract information from the following input: Sally is 13 and favorite food is Korean food
Human: Tip: Make sure to answer in the correct format

> Finished chain.


Person(name='Sally', age=13, fav_food='Korean food')

In [14]:
chain = create_structured_output_chain(People, llm, prompt, verbose=True)
chain.run("Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for extracting information in structured formats.
Human: Use the given format to extract information from the following input: Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally.
Human: Tip: Make sure to answer in the correct format

> Finished chain.


People(people=[Person(name='Sally', age=13, fav_food=''), Person(name='Joey', age=12, fav_food='spinach'), Person(name='Caroline', age=23, fav_food='')])

In [15]:
json_schema = {
    "title": "Person",
    "description": "Identifying information about a person.",
    "type": "object",
    "properties": {
        "name": {"title": "Name", "description": "The person's name", "type": "string"},
        "age": {"title": "Age", "description": "The person's age", "type": "integer"},
        "fav_food": {
            "title": "Fav Food",
            "description": "The person's favorite food",
            "type": "string",
        },
    },
    "required": ["name", "age"],
}
chain = create_structured_output_chain(json_schema, llm, prompt, verbose=True)
chain.run("Sally is 13")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for extracting information in structured formats.
Human: Use the given format to extract information from the following input: Sally is 13
Human: Tip: Make sure to answer in the correct format

> Finished chain.


{'name': 'Sally', 'age': 13}

### Caching

In [8]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

conversation.run("Answer briefly. What are the first 3 colors of a rainbow?")
conversation.run("And the next 4?")

'The next four colors of a rainbow are green, blue, indigo, and violet.'

### Async API
- asyncio 라이브러리를 통한 체인의 비동기 지원

In [6]:
import asyncio
import time

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


def generate_serially():
    llm = OpenAI(temperature=0.9)
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    for _ in range(5):
        resp = chain.run(product="baseball equipment")
        print(resp)


async def async_generate(chain):
    resp = await chain.arun(product="baseball equipment")
    print(resp)


async def generate_concurrently():
    llm = OpenAI(temperature=0.9)
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    tasks = [async_generate(chain) for _ in range(5)]
    await asyncio.gather(*tasks)


s = time.perf_counter()
await generate_concurrently()
elapsed = time.perf_counter() - s
print("\033[1m" + f"Async function executed in {elapsed:0.2f} seconds." + "\033[0m")

s = time.perf_counter()
generate_serially()
elapsed = time.perf_counter() - s
print("\033[1m" + f"Serial executed in {elapsed:0.2f} seconds." + "\033[0m")



Diamond Sports Gear.


Diamond Glove Sports.


Hit & Field Supplies.


Batty's Baseball Supplies


Batter's Edge.
Async function executed in 0.84 seconds.


Diamond Sports.


Diamond Sports Gear.


Batters and Blades.


Diamond Bat Company.


PowerPlay Baseball Gear.
Serial executed in 2.72 seconds.


## Foundational

### LLM

In [5]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "What is a good name for a company that makes {product}?"

llm = OpenAI(temperature=0.9)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain("Korean food")

{'product': 'Korean food', 'text': '\n\nK-Food Kitchen'}

In [6]:
input_list = [
    {"product": "socks"},
    {"product": "computer"},
    {"product": "shoes"}
]

llm_chain.apply(input_list)

[{'text': '\n\nSocktastic.'},
 {'text': '\n\nMystic Computers'},
 {'text': '\n\nSole Solutions Footwear.'}]

In [7]:
llm_chain.generate(input_list)

LLMResult(generations=[[Generation(text='\n\nCozyToes Socks.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nBlueByte Technologies.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nStepStrong Shoes.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'completion_tokens': 21, 'total_tokens': 57, 'prompt_tokens': 36}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('8590805f-64e6-4384-9b1e-a8a8a8b994a1')), RunInfo(run_id=UUID('77565b76-5103-4ff9-a2b6-10450ad332dd')), RunInfo(run_id=UUID('b4d144f6-528f-4793-bd1b-b12f62bab7f0'))])

In [8]:
# Single input example
llm_chain.predict(product="colorful socks")

'\n\nMiniMism SockCo.'

In [9]:
# Multi inputs in tempalte
template = """Tell me a {adjective} joke about {subject}."""

llm = OpenAI(temperature=0.9)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(template)
)
llm_chain.predict(adjective="sad", subject="ducks")

"\n\nQ: What did the duck say when his car broke down?\nA: Quack me up, I'm feeling down."

In [10]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all the colors in a rainbow"""
llm_chain = LLMChain(prompt=PromptTemplate.from_template(template), llm=llm, output_parser=output_parser)
# llm_chain = LLMChain(prompt=PromptTemplate(template=template, output_parser=output_parser), llm=llm)

llm_chain.predict()

['Red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet']

## Router

In [11]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [12]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

In [13]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

llm = OpenAI()

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [14]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [15]:
print(chain.run("What is black body radiation?"))



> Entering new MultiPromptChain chain...


d:\Gang\study\Practice-LLM\llm_env\lib\site-packages\langchain\chains\llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is black body radiation?'}
> Finished chain.


Black body radiation is the electromagnetic radiation emitted from an idealized physical body that is in thermal equilibrium (that is, at a constant temperature) with its environment. This radiation is emitted across the entire electromagnetic spectrum, and is characteristic of the temperature of the body. The spectrum of the radiation is determined by the temperature of the body, which is known as the Planck spectrum.


In [16]:
print(
    chain.run(
        "What is the first prime number greater than 40 such that one plus the prime number is divisible by 3?"
    )
)



> Entering new MultiPromptChain chain...
math: {'input': 'What is the first prime number greater than 40 such that one plus the prime number is divisible by 3?'}
> Finished chain.


Answer: 43
